SIDE NOTE: would love to get any critiques, hints, tips, oberservations on any level not just on ML (but obviously ML as the priority).  Beginner Python learner (though I took the Full Stack Course) so any suggestions there are welcome.  Thanks.

Will need to import these libs plus setting some global variables

In [1]:
import sys
import os
import numpy as np
import renders as rs
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from IPython.display import display # Allows the use of display() for DataFrames
from sklearn.externals import joblib
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.cross_validation import StratifiedKFold
from sklearn.svm import SVC
import matplotlib.cm as cm
# Show matplotlib plots inline (nicely formatted in the notebook)
%matplotlib inline

# Might need to change the path of the included libraries.
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/match_stats.py')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/form_model.py')
sys.path.append('/anaconda/envs/stats/lib/python3.5/site-packages')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/model_libs.py')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/form_data.py')
sys.path.append('/Users/senzari/Machine_Learning/stats')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats')
# print(sys.path)
os.chdir('/Users/senzari/Machine_Learning/stats/src')
#print(os.getcwd())

from stats import form_data, match_stats, model_libs, form_model, predict_matches

# Variables
round_number = 27 # for MLS only
target_col = 'points'
ignore_cols = ['games_played', 'match_id', 'team_id', 'team_name', 'opp_id', 'opp_name', 'scheduled']
all_models = ['log', 'svc', 'gmm', 'knn', 'gnb']

""" this variable 'testing' should be false if using CSV's and not pulling from the database. """
testing = False

INITIALIZED...


Pulling in the data either from the Database or the CSV (CSV for testers).  Data has some added features in between the database and 'raw_data'.  Essentially trying 3 different versions of the data in this set.  One version contains all the features.  Another version has the standard features plus the calculated features of the home/away stats for a matchup.  The last version has the standard features plus the 'extended features' which not only includes the stats of the two teams in the match up but also the current teams previous opponents and the current teams opponents of the team opponents in that matchup.

In [2]:
data_csv = 'round' + str(round_number) + '.csv'

if testing:
    raw_data = form_data.run_data(round_number)
    raw_data.to_csv(data_csv)
    print("Raw Data Saved to CSV")
else:
    #Reading in a CSV adds the first index column
    raw_data = pd.read_csv(data_csv)
    raw_data = raw_data.drop(raw_data.columns[[0]], axis=1)

pd.set_option("display.max_columns", 85)

print('Data Loaded...')

Data Loaded...


## Training

Functions that trains basic models (and saves them if need be).  'load_models' will  load exisiting models to save time in testing.

In [3]:
# Removes Columns to Ignore and Splits the Target Column
def split_target(data):
    td = model_libs._clone_and_drop(data, ignore_cols)
    (y, X) = model_libs._extract_target(td, target_col)
    return X, y

"""No Cross-Validation and No Tuning"""
def train_models(round_num, X, y, models):
    
    if os.path.isdir("/models/" + str(round_num)):
        print('Making New Directory for the Round')
        os.chdir('/Users/senzari/Machine_Learning/stats/src/models')
        os.makedirs(str(round_num))
        os.chdir('/Users/senzari/Machine_Learning/stats/src')
        
    finished_models = []
    
    for i in models:
        
        model_round = 'models/' + str(round_num) + '/' + str(i) + '_round_' + str(round_num) + '.pk1'
        
        
        if i == 'log':
            log = form_model.build_model(X, y, i)
            joblib.dump(log, model_round)
            finished_models.append(log)
        elif i == 'svc':
            svc = form_model.build_model(X, y, i)
            joblib.dump(svc, model_round)
            finished_models.append(svc)
        elif i == 'gmm':
            gmm = form_model.build_model(X, y, i)
            joblib.dump(gmm, model_round)
            finished_models.append(gmm)
        elif i == 'knn':
            kmeans = form_model.build_model(X, y, i)
            joblib.dump(kmeans, model_round)
            finished_models.append(kmeans)
        elif i == 'gnb':
            gnb = form_model.build_model(X, y, i)
            joblib.dump(gnb, model_round)
            finished_models.append(gnb)
    
    return finished_models
        
        
def load_models(round_num):
    
    loaded_models = []
    
    for i in all_models:
        model_round = 'models/' + str(round_num) + '/' + str(i) + '_round_' + str(round_num) + '.pk1'
        if i == 'log':
             log = joblib.load(model_round)
             loaded_models.append(log)
        if i == 'svc':
             svc= joblib.load(model_round)
             loaded_models.append(svc)
        elif i == 'gmm':
            gmm = joblib.load(model_round)
            loaded_models.append(gmm)
        elif i == 'knn':
            kmeans = joblib.load(model_round)
            loaded_models.append(kmeans)
        elif i == 'gnb':
            gnb = joblib.load(model_round)
            loaded_models.append(gnb)
        
        print("Success :: Loaded - " + str(i))
        
    return loaded_models

# raw_X, raw_y = split_target(raw_data)

# display(raw_data.head())

pd.set_option('display.max_columns', 500)

formatted_data = raw_data.drop('points', 1)

def set_group(goals):
    if goals >= 2:
        return 1
    elif goals < 2:
        return 0
    
formatted_data['converted_goals'] = formatted_data.apply(lambda row: set_group(row['goals']), axis=1)

all_data = formatted_data

formatted_data = model_libs._clone_and_drop(formatted_data, ignore_cols)

rf = model_libs.reformat_formation(formatted_data, formatted_data.groupby('current_formation').groups, formatted_data.groupby('opp_formation').groups)

for r in range(len(rf)):
    formatted_data = formatted_data.replace(rf[r], r)

partially_formatted_data = formatted_data
formatted_data = formatted_data.drop('goals', 1)

(f_y, f_X) = model_libs._extract_target(formatted_data, 'converted_goals')
    
#display(formatted_data.head())

### FORMATTING DATA AS NEEDED

In [ ]:
standard_cols = ['current_formation', 'is_home', 'avg_points', 'avg_goals_for', 'avg_goals_against', 'margin', 'goal_diff',
                'win_percentage', 'sos', 'opp_is_home', 'opp_avg_points', 'opp_avg_goals', 'opp_margin', 'opp_goal_diff', 'opp_win_percentage',
                'opp_opp_record', 'goals']
# 17 in each
home_cols = ['current_team_home_possession', 'current_team_home_attacks', 'current_team_home_dangerous_attacks', 'current_team_home_yellow_card',
            'current_team_home_corner_kicks', 'current_team_home_shots_on_target', 'current_team_home_shots_total', 'current_team_home_ball_safe',
            'current_team_home_played', 'current_opp_away_attacks', 'current_opp_away_dangerous_attacks', 'current_opp_away_yellow_card',
            'current_opp_away_corner_kicks', 'current_opp_away_shots_on_target', 'current_opp_away_shots_total', 'current_opp_away_ball_safe',
            'current_opp_away_played']
away_cols = ['current_team_away_possession', 'current_team_away_attacks', 'current_team_away_dangerous_attacks', 'current_team_away_yellow_card', 
            'current_team_away_corner_kicks', 'current_team_away_shots_on_target', 'current_team_away_shots_total', 'current_team_away_ball_safe', 
            'current_team_away_played', 'current_opp_home_attacks', 'current_opp_home_dangerous_attacks', 'current_opp_home_yellow_card', 'current_opp_home_corner_kicks', 
             'current_opp_home_shots_on_target', 'current_opp_home_shots_total', 'current_opp_home_ball_safe', 'current_opp_home_played']

extended_features = ['e_f_dangerous_attacks', 'e_f_shots_total', 'e_f_shots_on_target', 'e_f_ball_safe', 'e_f_possession', 'e_f_attacks',
               'opp_e_f_dangerous_attacks', 'opp_e_f_shots_total', 'opp_e_f_shots_on_target', 'opp_e_f_ball_safe', 'opp_e_f_possession', 'opp_e_f_attacks',
               'prev_opp_e_f_dangerous_attacks', 'prev_opp_e_f_shots_total', 'prev_opp_e_f_shots_on_target', 'prev_opp_e_f_ball_safe', 'prev_opp_e_f_possession', 'prev_opp_e_f_attacks',
               'opp_opp_e_f_dangerous_attacks', 'opp_opp_e_f_shots_total', 'opp_opp_e_f_shots_on_target', 'opp_opp_e_f_ball_safe', 'opp_opp_e_f_possession', 'opp_opp_e_f_attacks']

optional_extended_features = ['prev_opp_e_f_dangerous_attacks', 'prev_opp_e_f_shots_total', 'prev_opp_e_f_shots_on_target', 'prev_opp_e_f_ball_safe', 'prev_opp_e_f_possession', 'prev_opp_e_f_attacks',
               'opp_opp_e_f_dangerous_attacks', 'opp_opp_e_f_shots_total', 'opp_opp_e_f_shots_on_target', 'opp_opp_e_f_ball_safe', 'opp_opp_e_f_possession', 'opp_opp_e_f_attacks']

played_features = ['current_team_home_played', 'current_team_away_played','current_opp_home_played', 'current_opp_away_played', 'opp_is_home']


### RUNNING CLASSIFICATION MODEL ON IF TEAMS SCORE 0-1 OR 1-2 ON GAMES

In [ ]:
def run_features(data, drop_data, target, models):
    
    new_data = data.drop(drop_data, axis=1)
    
    display(new_data.head())
    
    (y, X) = model_libs._extract_target(new_data, target)
    
    models = train_models(round_number, X, y, models)
    
    return models


#### Running ALL Features 
#models_test_1 = run_features(formatted_data, [], 'converted_goals', all_models)

#### Running only STANDARD Features
#models_test_2 = run_features(formatted_data, home_cols + away_cols + extended_features + played_features, 'converted_goals', all_models)

#### Running STANDARD and EXTENDED Features
#models_test_3 = run_features(formatted_data, home_cols + away_cols + optional_extended_features + played_features, 'converted_goals', all_models)

# Creates new column list for merged columns
merged_cols = []

for cols in home_cols:
    merged_cols.append(model_libs.rename_column(cols))
    
# Creates a Home and Away Table with Standard and Home/Away Columns.  Will merge together further down
#ha_data = model_libs._clone_and_drop(formatted_data, ignore_cols)
#display(reg_data.head())
home_data = formatted_data.loc[formatted_data.loc[:, 'is_home'] == 1, standard_cols + home_cols + extended_features + ['converted_goals']]
#display(home_data.head())
away_data = formatted_data.loc[formatted_data.loc[:, 'is_home'] == 0, standard_cols + away_cols + extended_features + ['converted_goals']]

# Combine the Home Table with the Away Table
ha_data = home_data.append(away_data)

ha_data = ha_data.drop('goals', axis=1)

# Combine Home/Away Columns into one for Current Team and Opponent.  Cuts down on Features
# Adds from whatever column that doesn't have Nan
for c in range(len(merged_cols)):
    ha_data[merged_cols[c]] = ha_data.apply(lambda row: model_libs.pick_column(row[home_cols[c]], row[away_cols[c]]), axis=1 )

ha_data = ha_data.drop(home_cols + away_cols, axis=1)

#### Running STANDARD and HOME and AWAY Features
#models_test_4 = run_features(ha_data, extended_features + ['current_opp_played', 'current_team_played'], 'converted_goals', ['log'])


models_test_1 = run_features(formatted_data, [], 'converted_goals', ['log'])
(f_y, f_X) = model_libs._extract_target(formatted_data, 'converted_goals')
log = models_test_1[0]
print(log.predict_proba(f_X))

actual_y = pd.DataFrame(f_y.values, columns=['actual'])
predictions = pd.DataFrame(log.predict(f_X), columns=['predictions'])
preds = pd.concat([predictions, actual_y], axis=1)
preds['diff'] = preds.apply(lambda r: model_libs.predictions_diff(r['predictions'], r['actual']), axis=1)
accuracy = np.divide(preds['diff'].sum(),float(len(preds['diff'])))
display(preds.head())
print(accuracy)


# Optimize Features

Since STANDARD and HOME and AWAY Features got the best score going to optimize and minimize off of those features.

In [ ]:
def plot_RFECV(X, y):
    svc = SVC(kernel="linear")
    rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(y, 2),
                  scoring='accuracy')
    rfecv.fit(X, y)

    # Plot number of features VS. cross-validation scores
    plt.figure()
    plt.xlabel("Number of features selected")
    plt.ylabel("Cross validation score (nb of correct classifications)")
    plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
    plt.show()
    
plot_RFECV(ha_X, ha_y)

In [ ]:
def create_RFE(X, y):
    # Create the RFE object and rank each pixel
    svc = SVC(kernel="linear", C=1)
    rfe = RFE(estimator=svc, n_features_to_select=1, step=1)
    rfe.fit(X, y)
    print(rfe.ranking_)
    return rfe.ranking_

(ha_y, ha_X) = model_libs._extract_target(ha_data, 'converted_goals')

# Taking all of Home/Away Data (even the Extended Features)
rankings = create_RFE(ha_X, ha_y)

In [ ]:
""" Reordering the columns in the data to reflect the rankings """

rankings_name = []
ranked_X = ha_X

for r in range(len(rankings)):
    column_name = ha_X.iloc[:, rankings[r]-1].name
    rankings_name.append(column_name)

ranked_X = ranked_X.reindex_axis(rankings_name, axis=1)
display(ranked_X.head())

In [ ]:
def train_ranked_models(X, y):
    
    for x in xrange(10, 20):
        print('X :: {}'.format(x))
        ranked_models = train_models(round_number, X.iloc[:, 1:x], y, ['knn'])
    
train_ranked_models(ranked_X, ha_y)

In [ ]:
def plot_PCA(data, preds):
    
    pca = PCA(n_components=4)
    pca.fit(data)
    
    # Generate PCA results plot
    pca_results = rs.pca_results(data, pca)
    #print(pca_results)
    pca_data = pca.transform(data)
    
    # Create a DataFrame for the reduced data
    pca_data = pd.DataFrame(pca_data, columns = ['Dimension 1', 'Dimension 2', 'Dimension 3', 'Dimension 4'])
    plot_data = pd.concat([preds, pca_data], axis = 1)
    
    # Scatterplotting the transformed data if it's 2 Dimensions
    """fig, ax = plt.subplots(figsize = (14,8))
    cmap = cm.get_cmap('gist_rainbow')
    
    for i, cluster in plot_data.groupby('converted_goals'):   
        cluster.plot(ax = ax, kind = 'scatter', x = 'Dimension 1', y = 'Dimension 2', 
                     color = cmap((i)*1.0/(4)), label = 'Points %i'%(i), s=30);"""
    
    return pca, plot_data, pca_results

#(ha_y, ha_X) = model_libs._extract_target(ha_data, 'converted_goals') 

pca, pca_data, pca_results = plot_PCA(ha_X, ha_y)
#display(pca_data.head())

print(pca_data.shape)
# Runs models on PCA to see if there is a difference in the results
models = run_features(pca_data, [], 'converted_goals', ['knn'])
(pca_y, pca_X) = model_libs._extract_target(pca_data, 'converted_goals')
knn = models[0]
#preds = knn.predict(pca_X)

actual_y = pd.DataFrame(pca_y.values, columns=['actual'])
predictions = pd.DataFrame(knn.predict(pca_X), columns=['predictions'])
preds = pd.concat([predictions, actual_y], axis=1)
preds['diff'] = preds.apply(lambda r: model_libs.predictions_diff(r['predictions'], r['actual']), axis=1)
accuracy = np.divide(preds['diff'].sum(),float(len(preds['diff'])))
print(accuracy)


In [ ]:
display(pca_results)
print(type(pca_results))
print(pca_results[])

### Making Predictions on Final Selected Model

In [ ]:
(ha_y, ha_X) = model_libs._extract_target(ha_data, 'converted_goals')
knn = train_models(round_number, ha_X, ha_y, ['knn'])
actual_y = pd.DataFrame(ha_y.values, columns=['actual'])

predictions = pd.DataFrame(knn[0].predict(ha_X), columns=['expected_goal_category'])
preds = pd.concat([predictions, actual_y], axis=1)
preds['diff'] = preds.apply(lambda r: model_libs.predictions_diff(r['expected_goal_category'], r['actual']), axis=1)
accuracy = np.divide(preds['diff'].sum(),float(len(preds['diff'])))
print(accuracy)

======================================================================================================================

### RUNNING LINEAR REGRESSION on FEATURES

### Creating Regression Model independent from Classification (without the Classifier's input as a feature)

In [14]:
# Can't use formatted_data.  Still need the goals column
regr_data = all_data
#display(regr_data)
g = regr_data.groupby(['match_id'])
groups = dict(list(g))
for g in groups:
    print(' ========== ')
    print(g)
    teams = np.array(groups[g]['team_id'].values)
    opps = np.array(groups[g]['opp_id'].values)
    for t, o in zip(teams, opps):    
        print('t : {}, o : {}').format(t, o)
        match = regr_data[(regr_data['team_id'] == t) & (regr_data['opp_id'] == o)]
        match_id = match['match_id']
        curr_goals = match['converted_goals']
        reverse_match = regr_data[(regr_data['team_id'] == o) & (regr_data['opp_id'] == t)]
        reverse_match = reverse_match[reverse_match['match_id'] == match_id.values[0]]
        regr_data.loc[reverse_match.index.values[0], 'opp_converted_goals'] = curr_goals.values[0]
        
pd.set_option('display.max_rows', 5000)     



10.0
t : 62, o : 76.0
t : 76, o : 62.0
11.0
t : 41, o : 58.0
t : 58, o : 41.0
t : 64, o : 66.0
t : 66, o : 64.0
t : 86, o : 91.0
t : 91, o : 86.0
t : 105, o : 110.0
t : 110, o : 105.0
12.0
t : 42, o : 49.0
t : 49, o : 42.0
t : 63, o : 72.0
t : 72, o : 63.0
t : 82, o : 87.0
t : 87, o : 82.0
t : 117, o : 120.0
t : 120, o : 117.0
13.0
t : 45, o : 50.0
t : 50, o : 45.0
t : 65, o : 71.0
t : 71, o : 65.0
t : 95, o : 99.0
t : 99, o : 95.0
t : 116, o : 119.0
t : 119, o : 116.0
14.0
t : 48, o : 57.0
t : 57, o : 48.0
t : 75, o : 78.0
t : 78, o : 75.0
t : 89, o : 94.0
t : 94, o : 89.0
t : 114, o : 115.0
t : 115, o : 114.0
15.0
t : 44, o : 46.0
t : 46, o : 44.0
t : 69, o : 77.0
t : 77, o : 69.0
t : 88, o : 90.0
t : 90, o : 88.0
t : 109, o : 113.0
t : 113, o : 109.0
16.0
t : 53, o : 56.0
t : 56, o : 53.0
t : 67, o : 74.0
t : 74, o : 67.0
t : 97, o : 100.0
t : 100, o : 97.0
t : 103, o : 111.0
t : 111, o : 103.0
17.0
t : 54, o : 59.0
t : 59, o : 54.0
t : 68, o : 70.0
t : 70, o : 68.0
t : 83, o : 98.0

In [15]:
regr_data.to_csv('converted_goals.csv')
display(regr_data)

,match_id,team_id,team_name,opp_id,opp_name,scheduled,games_played,is_home,current_formation,avg_points,avg_goals_for,avg_goals_against,margin,goal_diff,win_percentage,sos,opp_is_home,opp_formation,opp_avg_points,opp_avg_goals,opp_margin,opp_goal_diff,opp_win_percentage,opp_opp_record,current_team_home_possession,current_team_away_possession,current_team_home_attacks,current_team_away_attacks,current_team_home_dangerous_attacks,current_team_away_dangerous_attacks,current_team_home_yellow_card,current_team_away_yellow_card,current_team_home_corner_kicks,current_team_away_corner_kicks,current_team_home_shots_on_target,current_team_away_shots_on_target,current_team_home_shots_total,current_team_away_shots_total,current_team_home_ball_safe,current_team_away_ball_safe,current_team_home_played,current_team_away_played,current_opp_home_attacks,current_opp_away_attacks,current_opp_home_dangerous_attacks,current_opp_away_dangerous_attacks,current_opp_home_yellow_card,current_opp_away_yellow_card,current_opp_home_corner_kicks,current_opp_away_corner_kicks,current_opp_home_shots_on_target,current_opp_away_shots_on_target,current_opp_home_shots_total,current_opp_away_shots_total,current_opp_home_ball_safe,current_opp_away_ball_safe,current_opp_home_played,current_opp_away_played,e_f_dangerous_attacks,e_f_shots_total,e_f_shots_on_target,e_f_ball_safe,e_f_possession,e_f_attacks,opp_e_f_dangerous_attacks,opp_e_f_shots_total,opp_e_f_shots_on_target,opp_e_f_ball_safe,opp_e_f_possession,opp_e_f_attacks,prev_opp_e_f_dangerous_attacks,prev_opp_e_f_shots_total,prev_opp_e_f_shots_on_target,prev_opp_e_f_ball_safe,prev_opp_e_f_possession,prev_opp_e_f_attacks,opp_opp_e_f_dangerous_attacks,opp_opp_e_f_shots_total,opp_opp_e_f_shots_on_target,opp_opp_e_f_ball_safe,opp_opp_e_f_possession,opp_opp_e_f_attacks,goals,converted_goals,opp_converted_goals
0,235,21,FC Dallas,38,Houston Dynamo,2016-03-13 01:30:00,1,0,4-2-3-1,3.000000,2.000000,0.000000,2.000000,2,1.000000,1.166667,1,4-2-3-1,1.000000,3.000000,0.000000,0,0.500000,0.500000,46.000000,48.195318,93.000000,96.025639,59.000000,46.561958,1.000000,1.916176,8.000000,4.184249,9.000000,4.131807,12.000000,8.097396,87.000000,92.561330,1,0,65.000000,96.025639,44.000000,46.561958,1.000000,1.916176,5.000000,4.184249,5.000000,4.131807,14.000000,8.097396,66.000000,92.561330,1,0,59.000000,12.000000,9.000000,87.000000,46.000000,93.000000,44.000000,14.000000,5.000000,66.000000,48.000000,65.000000,55.000000,5.000000,1.000000,91.000000,54.000000,92.000000,51.000000,13.000000,6.000000,75.000000,52.000000,74.000000,0,0,1
1,241,21,FC Dallas,29,Montreal Impact,2016-03-20 01:00:00,2,1,4-2-3-1,1.500000,1.000000,2.500000,-1.500000,-3,0.500000,2.000000,0,4-2-3-1,3.000000,3.000000,2.000000,4,1.000000,0.000000,46.000000,52.000000,93.000000,97.000000,59.000000,51.000000,1.000000,3.000000,8.000000,7.000000,9.000000,4.000000,12.000000,6.000000,87.000000,71.000000,1,1,119.000000,101.000000,45.000000,49.000000,1.000000,1.000000,3.000000,2.000000,7.000000,6.000000,10.000000,11.000000,119.000000,102.000000,1,1,55.000000,9.000000,6.500000,79.000000,49.000000,95.000000,47.000000,10.500000,6.500000,110.500000,49.500000,110.000000,44.000000,10.000000,5.000000,79.000000,48.000000,78.000000,68.500000,6.500000,1.500000,108.000000,46.000000,132.000000,2,1,0
2,249,21,FC Dallas,33,DC United,2016-03-26 21:30:00,3,0,4-2-3-1,2.000000,1.333333,1.666667,-0.333333,-1,0.666667,0.880952,1,4-4-2,0.666667,0.666667,-1.000000,-3,0.357143,0.500000,49.500000,52.000000,106.500000,97.000000,65.500000,51.000000,1.500000,3.000000,6.500000,7.000000,8.000000,4.000000,14.500000,6.000000,92.500000,71.000000,2,1,135.000000,79.000000,73.000000,38.500000,1.000000,3.500000,7.000000,4.500000,5.000000,3.000000,11.000000,5.500000,121.000000,81.000000,1,2,60.666667,11.666667,6.666667,85.333333,50.333333,103.333333,50.000000,7.333333,3.666667,94.333333,46.000000,97.666667,45.333333,8.666667,5.333333,112.333333,48.666667,113.666667,52.666667,9.333333,5.333333,96.666667,52.333333,1

In [ ]:
from sklearn import cross_validation
from sklearn.cross_validation import train_test_split, cross_val_predict
from sklearn import linear_model
from sklearn import grid_search
from sklearn import preprocessing
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_regression
from sklearn.cross_validation import cross_val_score

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)

# Can't use formatted_data.  Still need the goals column
regr_data = all_data

# Need to add results of the classifier to the features
regr_data = pd.concat([regr_data, predictions], axis=1)
display(regr_data.head())
rd = regr_data.groupby('current_formation')
print(rd)


# Both for current team and the current opponent
regr_data = model_libs._clone_and_drop(regr_data, home_cols + away_cols + extended_features + ignore_cols + ['points'])

# What other data can we remove that's not useful to predicting the number of goals per game
regr_data = regr_data.drop(['converted_goals'], axis=1)

(regr_y, regr_X) = model_libs._extract_target(regr_data, 'goals')

# Input X must be non-negative for chi-best one of the feautre selections. So will need to scale for this
# scaled_X = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(regr_X))

# Removes features with not enough variance - 80% is the threshold
#sel = VarianceThreshold(threshold = (.8 * (1 - .8)))
#sel_X = pd.DataFrame(sel.fit_transform(regr_X))

#display(regr_X.head())

# Selects the best features for regression
# k_X = SelectKBest(f_regression, k=20).fit_transform(regr_X, regr_y)
# display(pd.DataFrame(k_X))
        
X_train, X_test, y_train, y_test = cross_validation.train_test_split(regr_X, regr_y, test_size=0.2, random_state=42)

lin_regr = linear_model.LinearRegression()
lin_regr.fit(X_train, y_train)
#predicted = cross_val_predict(lin_regr, reg_X, reg_y, cv=10)
print('Coefficients: \n', lin_regr.coef_)
print("Residual sum of squares: %.2f"
      % np.mean((lin_regr.predict(X_test) - y_test) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % lin_regr.score(X_test, preds['rounded']))

"""regressor = DecisionTreeRegressor()
parameters = {'max_depth':(1,20,30,40,50,60,70,80,90,100), 'max_features': (2, 5, 10, 15, 20)}
dec_regr = grid_search.GridSearchCV(regressor, parameters, scoring=make_scorer(mean_squared_error, greater_is_better=False))
#print(cross_val_score(dec_regr, k_X, regr_y, cv=10))
print(dec_regr)
dec_regr.fit(X_train, y_train)"""

preds = pd.DataFrame(lin_regr.predict(X_test), columns=['predictions'])
preds['rounded'] = preds.loc[0:].apply(lambda r: np.round(r), axis=1)
display(preds.head())
preds = pd.concat([preds, pd.DataFrame(y_test.as_matrix(), columns=['actual'])], axis = 1)
preds['correct'] = preds.apply(lambda r: model_libs.predictions_diff(r['rounded'], r['actual']), axis=1)
preds['correct_category'] = preds.apply(lambda r: model_libs.check_category(r['rounded'], r['actual']), axis=1)
#display(preds.head())
accuracy = np.divide(preds['correct'].sum(),float(len(preds['correct'])))
accuracy_category = np.divide(preds['correct_category'].sum(), float(len(preds['correct_category'])))
print('accuracy on X TEST')
print(accuracy)
print('accuracy_category on X TEST')
print(accuracy_category)


Predicting all goals on the regression model

In [ ]:
k_X = pd.DataFrame(k_X)

all_preds = pd.DataFrame(lin_regr.predict(k_X), columns=['predictions'])
all_preds['rounded'] = all_preds.loc[0:].apply(lambda r: np.round(r), axis=1)

all_preds = pd.concat([k_X, all_preds], axis = 1)

all_preds = pd.concat([all_preds, regr_data['goals']], axis=1)

necessary = all_preds[['predictions', 'rounded', 'goals']]
    
necessary['correct'] = necessary.apply(lambda r: model_libs.predictions_diff(r['rounded'], r['goals']), axis=1)
necessary['correct_category'] = necessary.apply(lambda r: model_libs.check_category(r['predictions'], r['goals']), axis=1)
display(necessary)

accuracy = np.divide(necessary['correct'].sum(),float(len(necessary['correct'])))
accuracy_category = np.divide(necessary['correct_category'].sum(), float(len(necessary['correct_category'])))

print('accuracy on All of X')
print(accuracy)
print('accuracy_category on All of X')
print(accuracy_category)

### Part 4

Use these models trained on the optimal data to predict upcoming matches

Find upcoming matches...

In [ ]:
#upcoming_matches, match_details = predict_matches.get_upcoming_matches()
#upcoming_matches.to_csv('upcoming_matches.csv')
#match_details.to_csv('match_details.csv')
#print("Upcoming Data Saved to CSV")

upcoming_matches = pd.read_csv('upcoming_matches.csv')
upcoming_matches = upcoming_matches.drop(upcoming_matches.columns[[0]], axis=1)

match_details = pd.read_csv('match_details.csv')
match_details = match_details.drop(match_details.columns[[0]], axis=1)

display(upcoming_matches)

Make predictions on upcoming data

In [ ]:
from stats import predict_matches

for i in models:
    if i == 'svc':
        svc_preds, upcoming_data = predict_matches.predictions(upcoming_matches, match_details, svc)
        print(svc_preds)
    elif i == 'gmm':
        gmm_preds, upcoming_data = predict_matches.predictions(upcoming_matches, match_details, gmm)
        print(gmm_preds)
    elif i == 'knn':
        knn_preds, upcoming_data = predict_matches.predictions(upcoming_matches, match_details, knn)
        print(knn_preds)
    elif i == 'gnb':
        gnb_preds, upcoming_data = predict_matches.predictions(upcoming_matches, match_details, gnb)
        print(gnb_preds)
        
display(upcoming_data.head())

Output those predictions to a CSV so easier to visualize

In [ ]:
columns = ['team_name', 'opp_name', 'scheduled']
# Remove all columns except the ones above
upcoming_matches = upcoming_data[columns]

# Add predictions to the end of that DF
results = pd.DataFrame({'SVC': svc_preds, 'GMM': gmm_preds, 'KNN': knn_preds, 'GNB': gnb_preds})
upcoming_matches = upcoming_matches.join(results)
reordered_matches = pd.DataFrame([])

for rows in upcoming_matches.iterrows():
    for i in upcoming_matches['team_name']:
        if rows[1]['opp_name'] == i:
            reordered_matches = reordered_matches.append(rows[1])
            reordered_matches = reordered_matches.append(upcoming_matches[upcoming_matches['team_name'].isin([i])])

reordered_matches = reordered_matches.drop_duplicates() 
columns = ['scheduled', 'team_name', 'opp_name', 'SVC', 'KNN', 'GMM', 'GNB']
reordered_matches = reordered_matches[columns]
# upcoming_matches = upcoming_matches[(upcoming_matches['scheduled'] < '2016-08-26')]
reordered_matches.to_csv('predictions_' + str(round_number) + '.csv')
print('Prediction CSV saved')

### RESULTS

Excluding GMM...

FC Dallas vs Portland Timbers :: 3-1 (Unanimously Dallas)
New England Revolution vs Colorado Rapids :: 2-0 (Mixed, 1 model picked NER to win)
New York City FC vs DC United :: 3-2 (Unanimously NYCFC)
Chicago Fire vs Philadelphia Union :: 3-0 (2 models picked Chi and 1 picked Phi)
Vancouver Whitecaps vs New York Red Bulls :: 0-1 (WRONG, 2 models picked Vancouver to win)
LA Galaxy vs Columbus Crew :: 2-1 (2 models picked LA to win)

My Questions:
- For GMM, how can we tell what classification it's predicting.  If you notice in the predictions CSV it says 0, 1, 2.  How do I know which one of those are win, draw, or a loss (3, 1, 0)?